In [1]:
import mujoco
import mujoco.viewer
from nav2d import Nav2D
import numpy as np
import os
import matplotlib.pyplot as plt


model_json_path = os.path.join(os.getcwd(),"environment_params.json")
test_env = Nav2D(json_file= model_json_path)
model, data = test_env.model, test_env.data

vel_hist = []
avel_hist = []
angle_hist = []

Visualize the rendering using mujoco.viewer passive launch

In [ ]:
with mujoco.viewer.launch_passive(model, data) as viewer:
	i = 0
	
	# # Select the camera
	# CAMERA_NAME = "camera"
	# viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
	# viewer.cam.fixedcamid = model.camera(CAMERA_NAME).id

	# enable viewer options:
	viewer.opt.frame = mujoco.mjtFrame.mjFRAME_BODY
	viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

	while viewer.is_running():
		i+=1
		
		# --- control simulation
		ctrl = np.zeros(test_env.model.nv)
		ctrl[0] = 0.0005
		ctrl[2] = 0.005

		# --- simulation
		nobs, rew, term = test_env.step(ctrl)
		# print(test_env.data.qvel)
		viewer.sync()
		
		if (i % 100 == 0):
			# print(nobs[2])
			agent_pose, goal_pose, lidar_obs = test_env._get_obs()
			# print(agent_pose[0:3], goal_pose)

			# ============= agent control debug =============
			# x, y, theta, x_dot, y_dot, theta_dot = agent_pose
			# print(f"theta_{i:7d}: {agent_pose[2]/np.pi*180:6.3f} deg | theta_dot_{i:7d}: {agent_pose[5]/np.pi*180:6.2f} deg/s", end="\r")
			# print(f"Time {i:10d} - Robot position ({x:5.3f},{y:5.3f}) | Goal position {goal_pose}")
			# lin_vel = np.sqrt(x_dot**2 + y_dot**2)

			# angle_hist.append(theta)
			# vel_hist.append(lin_vel)
			# avel_hist.append(theta_dot)
			# print(f"Current robot linear velocities at time {i:7d}: {lin_vel:10.5}", end="\r")

			# ============= goal pose debug =============
			# print(goal_pose)

			# ============= LIDAR obs debug =============
			print(lidar_obs)

			# ============= termination debug =============
			# if term:
			# 	print(f"Termination condition reached at {i}")
			# 	ctrl[0] = 0.01

			# 	# pause the viewer
			# 	while viewer.is_running():
			# 		viewer.sync()
			# 	break

[0.9990576  1.00004346 1.00102072 1.00003486 0.9990576 ]
[0.99818319 1.00017511 1.00213212 1.0001402  0.99818319]
[0.99739135 1.00039471 1.00331929 1.00031593 0.99739135]
[0.99667362 1.000703   1.0045918  1.00056242 0.99667362]
[0.99603785 1.00109935 1.00594096 1.00087946 0.99603785]
[0.99548183 1.00158405 1.00736944 1.00126722 0.99548183]
[0.99500751 1.00215672 1.00887481 1.0017256  0.99500751]
[0.99461152 1.0028182  1.01046168 1.002255   0.99461152]
[0.9942893  1.00357028 1.01213725 1.00285627 0.9942893 ]
[0.99404703 1.0044117  1.01389373 1.00352906 0.99404703]
[0.99388705 1.00534181 1.01572801 1.00427325 0.99388705]
[0.9938024  1.00636381 1.01765165 1.00509024 0.9938024 ]
[0.99379716 1.00747678 1.01965947 1.00597985 0.99379716]
[0.99387531 1.00867907 1.02174538 1.00694161 0.99387531]
[0.9940309  1.00997447 1.02392078 1.0079772  0.9940309 ]
[0.9942686  1.01136093 1.02617833 1.00908601 0.9942686 ]
[0.99458678 1.01284027 1.02852249 1.01026901 0.99458678]
[0.99498487 1.01441387 1.030955

In [ ]:
figure = plt.figure(figsize=(32,9))
plt.plot(vel_hist)

figure = plt.figure(figsize=(32,9))
plt.plot(avel_hist)

# figure = plt.figure(figsize=(32,9))
# ctrl_x, ctrl_y, ctrl_z = zip(*ctrl_hist)
# linear_ctrl_norm = np.sqrt(np.square(ctrl_x) + np.square(ctrl_y))
# plt.plot(np.round(linear_ctrl_norm,4))

figure = plt.figure(figsize=(32,9))
plt.plot(angle_hist)

In [ ]:
figure = plt.figure(figsize=(32,9))
plt.plot(d_goal_hist)

In [ ]:
theta_range = np.linspace(0, 2*np.pi, num=100)
action = np.array([0.05, 0, 0], dtype=np.float32)
action_mag_hist = np.zeros(len(theta_range))

for idx, theta in enumerate(theta_range):
    rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]], dtype=np.float32)
    action_transformed = action
    action_transformed[:2] = rot_matrix @ action[:2]
    # print(action_transformed)
    action_mag_hist[idx] = np.linalg.norm(action_transformed[:2])

action_mag_hist